# <center>Проект: предсказание токсичности соединений</center>

**<u>Дано:</u>**

* `train.csv` – данные для обучения
* `test_only_smiles.csv` – данные для предсказания

**<u>Необходимо:</u>**

1) Проанализировать данные + визуализация
2) Предобработать данные, чтобы улучшить качество модели
3) Обучить модель и провалидировать ее (метрика: RMSE)
4) Проверить модель на *test* данных и прислать результаты вместе с ноутбуком

## <center>Библиотеки и загрузка данных</center>

In [1]:
import pandas as pd
import numpy as np

In [3]:
train = pd.read_csv("data/train.csv")

In [5]:
train.head()

,SMILES,activity
0,OCC=C,5.0
1,CO/N=C(/C(=O)OC)\C1=C(COC2=C(C)C=CC=C2)C=CC=C1,94.0
2,COC(=O)NS(=O)(=O)C1=CC=C(N)C=C1,5.4
3,CCCCC(CC)C(O)=O,-8.2
4,COC(=O)C1=C(CC(C)C)C(C2=NCCS2)=C(N=C1C(F)F)C(F...,26.2


## <center>Анализ данных</center>